# Spatial Heat Flow Fluctuations from Sources and Conductivity

In [ ]:
import numpy as np
from plotconfig import *
import scipy.sparse as sp
from cmcrameri.cm import *
from cache import cached_call
import matplotlib.pyplot as plt
from cmasher import get_sub_cmap
from scipy.optimize import minimize
from scipy.sparse.linalg import spsolve
from scipy.interpolate import RectBivariateSpline

### The Stationary Heat Equation
The stationary heat equation reads
$$
\partial_t u = \frac{1}{c\rho} \vec{\nabla} \big(k\vec{\nabla} T\big)  + \frac{q}{c\rho} = 0
$$
and, simplified:
$$
\big(\vec{\nabla} k \big) \cdot \big(\vec{\nabla} T \big) + k \Delta v = - q
$$
Expressed as an operator on $T$:
$$
\underbrace{\Big(\big(\vec{\nabla} k \big) \cdot \vec{\nabla} + k \Delta\Big)}_A T = - q
$$

Some parameter ranges:

##### Thermal Conductivity
Within Upper Rhine Graben: $1.0\leq k \leq 4.0$ (Harlé *et al*., 2019)

##### Heat Production
In cratonic rock (Austrail, Finland, Russia): $0\leq \dot{q}_V \leq 12\,\mathrm{µW\,m}^{-3}$. A good upper limit seems $8\,\mathrm{µW\,m}^{-3}$. (Jaupart & Mareschal, 2004, Fig. 1)

In [ ]:
def generate_design_matrix(x, y, dx, dy, k, periodic):
    
    nx = x.size
    ny = y.size
    n = nx*ny
    
    if ny < 4:
        raise RuntimeError("Not enough columns (need at least 4)")
    
    # Gradient of k:
    kxy = k.reshape((ny,nx))
    spl = RectBivariateSpline(x, y, kxy.T)
    dkdx = spl(x,y,dx=1).T
    dkdy = spl(x,y,dy=1).T
    dkdx[0,:] = dkdx[-1,:] = dkdx[:,0] = dkdx[:,-1] = 0
    dkdy[0,:] = dkdy[-1,:] = dkdy[:,0] = dkdy[:,-1] = 0
    assert dkdy.shape == (ny,nx)
    if np.all(kxy == k[0]):
        dkdy[...] = 0.0
        dkdx[...] = 0.0
    dkdx = dkdx.reshape(-1)
    dkdy = dkdy.reshape(-1)
    
    A = sp.lil_matrix((n,n))
    idx2 = 1.0 / dx**2
    idy2 = 1.0 / dy**2
    for i in range(n):
        # Central difference Laplace operator for diffusion
        # term:
        if periodic:
            A[i,i] = -2*k[i]*(idx2 + idy2)
            if i>=1:
                A[i,i-1] = k[i]*idx2 - 0.5 * dkdx[i] / dx 
            if i < n-1:
                A[i,i+1] = k[i]*idx2 + 0.5 * dkdx[i] / dx 
        else:
            if (i % nx) >= 1 and (i % nx) < nx-1:
                # Have derivative in x direction:
                if (i % nx) >= 2 and (i % nx) < nx-2:
                    A[i,i-2] =            1/12 * dkdx[i] / dx 
                    A[i,i-1] = k[i]*idx2 - 2/3 * dkdx[i] / dx 
                    A[i,i+1] = k[i]*idx2 + 2/3 * dkdx[i] / dx 
                    A[i,i+2] =           -1/12 * dkdx[i] / dx 
                else:
                    A[i,i-1] = k[i]*idx2 - 0.5 * dkdx[i] / dx 
                    A[i,i+1] = k[i]*idx2 + 0.5 * dkdx[i] / dx 
                if i < nx:
                    # y is at bottom boundary, where we use second-order
                    # accurate forward difference (for the Laplace operator)
                    # and third-order forward difference (for gradient):
                    #A[i,i]      = - 2*k[i]*(idx2 + idy2) - 11/6 * dkdy[i] / dy
                    #A[i,i+nx]   =   5*k[i]*idy2          +    3 * dkdy[i] / dy
                    #A[i,i+2*nx] =  -4*k[i]*idy2          -  1.5 * dkdy[i] / dy
                    #A[i,i+3*nx] =   1*k[i]*idy2          +  1/3 * dkdy[i] / dy
                    # at bottom boundary, keep temperature steady:
                    A[i,i] = -1.0
                else:
                    # y not at bottom boundary.
                    # Need only to check if we are at top boundary,
                    # where follow up element is kept at 0 temperature.
                    # Derivative in x and y direction:
                    if i < n-nx:
                        if i >= 2*nx and i < n-2*nx:
                            A[i,i]      = - 2*k[i]*(idx2 + idy2)
                            A[i,i-2*nx] =            1/12 * dkdy[i] / dy 
                            A[i,i-nx]   = k[i]*idy2 - 2/3 * dkdy[i] / dy 
                            A[i,i+nx]   = k[i]*idy2 + 2/3 * dkdy[i] / dy 
                            A[i,i+2*nx] =           -1/12 * dkdy[i] / dy 
                        else:
                            A[i,i]    = - 2*k[i]*(idx2 + idy2)
                            A[i,i-nx] = k[i]*idy2 - 0.5 * dkdy[i] / dy 
                            A[i,i+nx] = k[i]*idy2 + 0.5 * dkdy[i] / dy 
                    else:
                        # If we are at top boundary, need to use a modified formula
                        # to obtain the zero temperature on surface boundary condition.
                        # We employ a second-order difference scheme (from Cameron Taylor's
                        # "Finite Difference Coefficients Calculator"):
                        #  f'' = −1/12 * f2(-3) + 4/12 * f2(-2) + 1/2 * f2(-1) - 5/3 * f2(0) + 11/12 * f2(1)
                        # on the *half* spaced grid ('f2'). Then, the value f2(1) is exactly the boundary
                        # with f2(1)=0. Also f2(2)=0. Then we approximate f2(-1) = 0.5*(f2(-2) + f2(0)),
                        # f2(-3)=(f2(-4)+f2(-2))/2 and express the result in terms of
                        # the original grid spacing:
                        #   f'' = (-f(-2) + 13*f(-1) - 34*f(0))/(24*dy**2)
                        A[i,i]      = - k[i]*(2*idx2 - 17/12*idy2)
                        A[i,i-nx]   = 13/24*k[i]*idy2
                        A[i,i-2*nx] = -1/24*k[i]*idy2
                        
            else:
                # x on boundary.
                # Derivative only in y direction:
                if i < nx:
                    # y is also at bottom boundary, where we use second-order
                    # accurate forward difference:
                    #A[i,i]      = - 2*k[i]*idy2 - 11/6 * dkdy[i] / dy
                    #A[i,i+nx]   =   5*k[i]*idy2 +    3 * dkdy[i] / dy
                    #A[i,i+2*nx] =  -4*k[i]*idy2 -  1.5 * dkdy[i] / dy
                    #A[i,i+3*nx] =   1*k[i]*idy2 +  1/3 * dkdy[i] / dy
                    # at bottom boundary, keep temperature steady:
                    A[i,i] = -1.0
                else:
                    # y not at bottom boundary.
                    # Need only to check if we are at top boundary,
                    # where follow up element is kept at 0 temperature.
                    if i < n-nx:
                        A[i,i]    = -2*k[i]*idy2
                        A[i,i-nx] =    k[i]*idy2 - 0.5 * dkdy[i] / dy
                        A[i,i+nx] =    k[i]*idy2 + 0.5 * dkdy[i] / dy
                    else:
                        # Top boundary, see above.
                        A[i,i]      = - 17/12*k[i]*idy2
                        A[i,i-nx]   =   13/24*k[i]*idy2
                        A[i,i-2*nx] =   -1/24*k[i]*idy2
            
            
    return A.tocsc()

In [ ]:
def surface_heat_flow(T, k, dy):
    """
    Computes the surface heat flow.
    """
    ny = T.shape[0]
    nx = T.shape[1]
    print("(ny,nx):",(ny,nx))
    kxy = k.reshape((ny,nx))
    # compute the gradient:
    #A[i,i]      = - 2*k[i]*(idx2 + idy2) - 11/6 * dkdy[i] / dy
    #A[i,i+nx]   =   5*k[i]*idy2          +    3 * dkdy[i] / dy
    #A[i,i+2*nx] =  -4*k[i]*idy2          -  1.5 * dkdy[i] / dy
    #A[i,i+3*nx] =   1*k[i]*idy2          +  1/3 * dkdy[i] / dy
    grad_T = -1.0/dy * (11/6*T[-1,:] - 3*T[-2,:] + 1.5*T[-3,:] - 1/3*T[-4,:])
    
    # Then the heat flow:
    return kxy[-1,:] * grad_T

## Setting up the problem
### 1) Grid setup

In [ ]:
nx = 101
ny = 78
nx_f = 201
ny_f = 151

XMIN = -80000
XMAX = 120000
YMIN = -80000
YMAX = 0

Calculate the derived grid coordinates:

In [ ]:
dx = (XMAX - XMIN) / nx
x = XMIN + dx * (np.arange(nx) + 0.5)
dy = (YMAX - YMIN) / ny
y = YMIN + dy * (np.arange(ny) + 0.5)
xbounds = np.linspace(XMIN, XMAX, nx+1)
ybounds = np.linspace(YMIN, YMAX, ny+1)

print("dx:",dx)
print("dy:",dy)

X,Y = np.meshgrid(x,y)

Empty source term:

In [ ]:
qdot_T = np.zeros((ny,nx))

Homogeneous thermal diffusivity. We use the value of Lachenbruch & Sass (1980):
$$
    K = 2.5\,\mathrm{W}\,\mathrm{m}^{-1}\,\mathrm{K}^{-1}
$$

In [ ]:
K = 2.5
k = np.full((ny,nx), K)

## The Computational Class

In [ ]:
from scipy.interpolate import NearestNDInterpolator

In [ ]:
def generate_k_homogen(X,Y):
    return np.full(X.shape, K)

In [ ]:
def generate_k(X, Y):
    k = np.empty_like(X)
    k[...] = K - 1.1*np.exp(-((X - 6e3)**2    + (Y + 3e3)**2) / (5e3)**2) \
               - 1.1*np.exp(-((X - 25e3)**2 + (Y + 20e3)**2) / (5e3)**2) \
               - 0.7*np.exp(-((X - 30e3)**2 + (Y + 30e3)**2) / (15e3)**2) \
               - 1.1*np.exp(-((X - 35e3)**2 + (Y + 10e3)**2) / (5e3)**2) \
               - 1.1*np.exp(-((X + 20e3)**2 + (Y + 5e3)**2) / (5e3)**2)
    
    return k

In [ ]:
def _solve_hf_2d_backend(k, T_bot, q_ids, qmax, q0, hf_dest, xmask, dy, A, tol, method='trust-constr', verbose=True):
    q = np.zeros_like(k)
    q[0,:] = T_bot
    if q0 is None:
        q0 = np.full(q_ids.size, 0.1*qmax)
    elif q0.size != q_ids.size:
        q0 = q0.flat[q_ids]
        
    if hf_dest.size != q_ids.size:
        hf_dest = hf_dest[xmask]

    solver = sla.factorized(A)
    def solve(q):
        return solver((-q).reshape(-1)).reshape(k.shape)
        
    def cost(qi):
        # Solve the steady state heat flow:
        q.flat[q_ids] = qi
        T_steady = solve(q)

        hf = surface_heat_flow(T_steady, k, dy)[xmask]
        _cost = np.sum((hf-hf_dest)**2) / hf_dest.mean()**2

        if verbose:
            print("cost:",_cost)
    
        return _cost
    

    res = minimize(cost, q0,
                   bounds = [(0,qmax)] * q_ids.size,
                   method = method,
                   tol=tol)
    print("res:",res)
    q.flat[q_ids] = res.x

    T = solve(q)
    hf = surface_heat_flow(T, k, dy)

    return q, T, hf

In [ ]:
class Grid2D:
    def __init__(self, xmin, xmax, ymin, ymax, nx, ny, xmin_use, xmax_use):
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.nx = nx
        self.ny = ny
        self.dx = (xmax - xmin) / nx
        self.x = xmin + self.dx * (np.arange(nx) + 0.5)
        self.dy = (ymax - ymin) / ny
        self.y = ymin + self.dy * (np.arange(ny) + 0.5)
        #xbounds = np.linspace(XMIN, XMAX, nx_f+1)
        #ybounds = np.linspace(YMIN, YMAX, ny_f+1)

        print("dx:",self.dx)
        print("dy:",self.dy)
        # The grids:
        self.X, self.Y = np.meshgrid(self.x, self.y)
        
        # Selecting the indices:
        self.xmask = (self.x >= xmin_use) & (self.x <= xmax_use)
        self.q_i = np.arange(int(np.argwhere(self.xmask)[0]), int(np.argwhere(self.xmask)[-1])+2)
        self.q_j = np.arange(int(0.8*ny), ny-1)
        self.q_ids = (self.q_i[np.newaxis,:] + nx * self.q_j[:,np.newaxis]).flatten()
        
    

class HeatFlow2D:
    def __init__(self, grid, generate_k):
        self.grid = grid
        self.k = generate_k(self.grid.X, self.grid.Y)
        
        self.A = generate_design_matrix(self.grid.x, self.grid.y, self.grid.dx, self.grid.dy,
                                        self.k.reshape(-1), False)
        self.solver = sla.factorized(self.A)
    
    def solve(self, q):
        return self.solver((-q).reshape(-1)).reshape(self.k.shape)
    
    def bottom_temperature(self, target_heat_flow):
        return target_heat_flow * (self.grid.y.max() - self.grid.y.min()) / K
    
    def surface_heat_flow(self, T):
        return surface_heat_flow(T, self.k, self.grid.dy)

    def initial_guess_q0(self, hf2d, q):
        x = hf2d.grid.X
        y = hf2d.grid.Y
        scale = hf2d.grid.x.size / self.grid.x.size
        interp = LinearNDInterpolator(np.stack((x.flat, y.flat), axis=1), (scale*q).flat)
        z = interp(np.stack((self.grid.X.flat, self.grid.Y.flat), axis=1))
        if np.any(np.isnan(z)):
            interp = NearestNDInterpolator(np.stack((x.flat, y.flat), axis=1), (scale*q).flat)
            mask = np.isnan(z)
            z[mask] = interp(np.stack((self.grid.X.flat[mask], self.grid.Y.flat[mask]), axis=1))
        mask = np.ones(z.shape, dtype=bool)
        mask[self.grid.q_ids] = False
        z[mask] = 0.0
        z = z.reshape(self.k.shape)
        z[0,:] = q[0,0]
        return z
    
    
    def solve_for_bottom_temperature(self, T_bot):
        q = np.zeros_like(self.k)
        q[0,:] = T_bot
        return self.solve(q)
        
    
    def solve_for_target_heat_flow(self, hf_dest, T_bot, q0=None, qmax=8e-6, tol=1e-3,
                                   method=None, verbose=True):
        
        return cached_call(_solve_hf_2d_backend, self.k, T_bot, self.grid.q_ids, qmax, q0, hf_dest,
                           self.grid.xmask, self.grid.dy, self.A, tol, method=method, verbose=verbose)

### 2) Boundary conditions

In [ ]:
grid = Grid2D(XMIN, XMAX, YMIN, YMAX, nx, ny, -50e3, 75e3)

In [ ]:
hf2d = HeatFlow2D(grid, generate_k)

In [ ]:
T_bot = hf2d.bottom_temperature(target_heat_flow=68.3e-3)
T_steady = hf2d.solve_for_bottom_temperature(T_bot)
hf = hf2d.surface_heat_flow(T_steady)

In [ ]:
# The optimization:
hf2d_2 = HeatFlow2D(grid, generate_k_homogen)
T_bot2 = hf2d_2.bottom_temperature(target_heat_flow=40e-3)
q2, T_steady2, hf2 = hf2d_2.solve_for_target_heat_flow(hf, T_bot2, qmax=8e-6, tol=1e-9,
                                                       method='trust-constr')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(q2[1:,:])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(T_steady2[1:,:])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(hf)
ax.plot(hf2)

## Fine Grid:

In [ ]:
grid_f = Grid2D(XMIN, XMAX, YMIN, YMAX, nx_f, ny_f, -50e3, 75e3)
hf2d_f = HeatFlow2D(grid_f, generate_k)
T_bot_f = hf2d_f.bottom_temperature(target_heat_flow=68.3e-3)
T_steady_f = hf2d_f.solve_for_bottom_temperature(T_bot_f)
hf_f = hf2d_f.surface_heat_flow(T_steady_f)

# The optimization:
hf2d_f2 = HeatFlow2D(grid_f, generate_k_homogen)
T_bot_f2 = hf2d_f2.bottom_temperature(target_heat_flow=40e-3)
q0_f = hf2d_f2.initial_guess_q0(hf2d_2, q2)

In [ ]:
q2_f, T_steady2_f, hf2_f = hf2d_f2.solve_for_target_heat_flow(hf_f, T_bot_f2, qmax=8e-6, tol=1e-9,
                                                              method='trust-constr', q0=q0_f)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
h = ax.imshow(q0_f[1:,:])
fig.colorbar(h)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
h = ax.imshow(q2_f[1:,:])
fig.colorbar(h)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.imshow(T_steady2_f[1:,:])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(hf_f)
ax.plot(hf2_f)

## Solve the heat flow anomaly:

In [ ]:
i_hfa = int(np.argmin(hf2d_f.grid.x**2))
j0_hfa = int(np.argwhere(hf2d_f.grid.y > -20e3)[0])
j1_hfa = int(np.argwhere(hf2d_f.grid.y > -10e3)[0])

In [ ]:
qfault = np.zeros((hf2d_f.grid.ny, hf2d_f.grid.nx))
qfault[j0_hfa:j1_hfa, i_hfa] = 0.98e3 / (2*(max(hf2d_f.grid.y[j1_hfa], hf2d_f.grid.y[j0_hfa])
                                            - min(hf2d_f.grid.y[j1_hfa], hf2d_f.grid.y[j0_hfa]) + hf2d_f.grid.dx))


T_steady_fault_0 = hf2d_f.solve(qfault)
hf_fault_0 = hf2d_f.surface_heat_flow(T_steady_fault_0)


T_steady_fault_1 = hf2d_f2.solve(qfault)
hf_fault_1 = hf2d_f2.surface_heat_flow(T_steady_fault_1)
#T_steady_fault_1 = spsolve(A_opt, (-qfault).flat).reshape(qdot_T.shape)
#hf_fault_1 = surface_heat_flow(T_steady_fault_1, k_opt)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(hf2d_f.grid.x, hf_fault_0)
ax.plot(hf2d_f.grid.x, hf_fault_1)

In [ ]:
hf2d_f.k.min()

### Combined plot:

In [ ]:
fig = plt.figure(dpi=300, figsize=(6.975,3.5))
#ax_bg = fig.add_axes((0,0,1,1))

# Setting 1:
ax_set1 = fig.add_axes((0.07, 0.6, 0.45, 0.39))
ax_set1.set_aspect('equal')
ax_set1.set_xlabel('Lateral offset from fault (km)')
ax_set1.set_ylabel('Depth (km)')
cset =\
  ax_set1.contour(1e-3*hf2d_f.grid.x, 1e-3*hf2d_f.grid.y, hf2d_f.k, cmap=bamako, linewidths=0.7, levels=4)
ax_set1.scatter(1e-3*hf2d_f.grid.x, 1e-3*hf2d_f.grid.y.min()*np.ones_like(hf2d_f.grid.x), marker='.',
                edgecolor='none', s=10, c='tab:orange')
ax_set1.plot([1e-3*hf2d_f.grid.x[i_hfa]]*2, [1e-3*hf2d_f.grid.y[j0_hfa], 1e-3*hf2d_f.grid.y[j1_hfa]],
             color='tab:red')
ax_set1.set_ylim(-81,0)
ax_set1.axvline(1e-3*xlim[0], color='k', linewidth=0.7, linestyle='-')
ax_set1.axvline(1e-3*xlim[1], color='k', linewidth=0.7, linestyle='-')
ax_set1.text(-75, -10, '(a)', fontsize=10)
ax_set1.text(100, -79, f'T={int(T_bot)} K', va='bottom', ha='center')
cset_labels = []
for i,l in enumerate(cset.levels[1:-1]):
    cset_labels.append("%2.1f" % (l,))
cset_handles = cset.legend_elements()[0][1:-1]
leg = ax_set1.legend(cset_handles, cset_labels, title='$k(\\vec{x})$\n($\\mathrm{W\,mK}^{-1}$)')
plt.setp(leg.get_title(), multialignment='center')

# Setting 2:
ax_set2 = fig.add_axes((0.07, 0.1, 0.45, 0.39))
cax = fig.add_axes((0.16, 0.28, 0.2, 0.03))
ax_set2.set_aspect('equal')
ax_set2.set_xlabel('Lateral offset from fault (km)')
ax_set2.set_ylabel('Depth (km)')
ax_set2.set_ylim(ax_set1.get_ylim())
ax_set2.set_xlim(ax_set1.get_xlim())
h = ax_set2.pcolormesh(1e-3*hf2d_f2.grid.x[hf2d_f2.grid.q_i],
                       1e-3*hf2d_f2.grid.y[hf2d_f2.grid.q_j],
                       1e6 * q2_f[hf2d_f2.grid.q_j.min():hf2d_f2.grid.q_j.max()+1,
                                  hf2d_f2.grid.q_i.min():hf2d_f2.grid.q_i.max()+1],
                       shading='nearest', cmap=get_sub_cmap(lajolla_r, 0.0, 0.8),
                       rasterized=True)
ax_set2.scatter(1e-3*hf2d_f2.grid.x, 1e-3*hf2d_f2.grid.y.min()*np.ones_like(hf2d_f2.grid.x),
                marker='.', edgecolor='none', s=10, c='tab:orange')
ax_set2.plot([1e-3*hf2d_f2.grid.x[i_hfa]]*2, [1e-3*hf2d_f2.grid.y[j0_hfa], 1e-3*hf2d_f2.grid.y[j1_hfa]],
             color='tab:red', label='Fault\ntrace')
ax_set2.axvline(1e-3*xlim[0], color='k', linewidth=0.7, linestyle='-')
ax_set2.axvline(1e-3*xlim[1], color='k', linewidth=0.7, linestyle='-')
ax_set2.text(-75, -10, '(b)', fontsize=10)
ax_set2.text(100, -79, f'T={int(T_bot2)} K', va='bottom', ha='center')
ax_set2.legend(loc='center right', bbox_to_anchor=(1.01, 0.5))
fig.colorbar(h, cax=cax, orientation='horizontal',
             label='Thermal power density\n($\mathrm{W\,m}^{-3}$)')

# Heat flow:
ax_hf = fig.add_axes((0.615, 0.61, 0.38, 0.33))
ax_hf.set_xlim(1e-3*xlim[0], 1e-3*xlim[1])
ax_hf.set_title('(c) Undisturbed heat flow')
ax_hf.set_xlabel('Lateral offset from fault (km)', labelpad=0.5)
ax_hf.set_ylabel('Heat flow ($\mathrm{mW\,m}^{-2}$)')
h0 = ax_hf.plot(1e-3*hf2d_f.grid.x, 1e3*hf_f, color='#aaaaaa', linewidth=2)
h1 = ax_hf.plot(1e-3*hf2d_f2.grid.x, 1e3*hf2_f, color='k', linestyle='--', linewidth=0.7)
ax_hf.legend(handles=(h0[0],h1[0]), labels=('(a)','(b)'))

# Effect onto the heat flow anomaly:
ax_anomaly = fig.add_axes((0.615, 0.096, 0.38, 0.33))
ax_anomaly.set_title('(d) Fault-generated anomaly')
ax_anomaly.set_xlim(1e-3*xlim[0], 1e-3*xlim[1])
ax_anomaly.set_xlabel('Lateral offset from fault (km)', labelpad=0.5)
ax_anomaly.set_ylabel('Heat flow ($\mathrm{mW\,m}^{-2}$)')
h0 = ax_anomaly.plot(1e-3*hf2d_f.grid.x, hf_fault_0, color='#aaaaaa', linewidth=2)
h1 = ax_anomaly.plot(1e-3*hf2d_f.grid.x, hf_fault_1, color='k', linestyle='--', linewidth=0.7)
ax_anomaly.legend(handles=(h0[0],h1[0]), labels=('(a)','(b)'))
fig.savefig('figures/A9-Sketch-HF-Fluctuation-Groups.pdf')

### References

> Jaupart C, Mareschal J-C (2003) Constraints on crustal heat production
>   from heat flow data. In: Rudnick R (ed) Treatise on geochemistry: the crust,
>   ch 2, vol 3. Elsevier, Amsterdam, pp 65–84
>
> Harlé, P., Kushnir, A. R. L., Aichholzer, C., Heap, M. J., Hehn, R., Maurer, V.,
>   Baud, P., Richard, A., Genter, A., and Duringer, P.: Heat flow density estimates
>   in the Upper Rhine Graben using laboratory measurements of thermal conductivity
>   on sedimentary rocks, Geothermal Energy, 7, 18, https://doi.org/10.1186/s40517-019-0154-3,
>   2019.

### License
```
A notebook to visualize the difference between spatial heat flow
fluctuations caused by inhomogeneous source density and inhomogeneous
thermal conductivity.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2022 Deutsches GeoForschungsZentrum Potsdam
            2022 Malte J. Ziebarth
            
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```